In [2]:
import pandas as pd
import pulp as lp
import numpy as np
from operator import iadd
from functools import reduce
from typing import Sequence, Any, DefaultDict, List

In [3]:
def create_prob(prob_name: str, sense: int) -> lp.LpProblem:
    return lp.LpProblem(prob_name, sense)


def add_obj_fn(lp_prob: lp.LpProblem, dvar: lp.LpAffineExpression) -> lp.LpProblem:
    return iadd(lp_prob, dvar)


def add_constraint(lp_prob: lp.LpProblem, constrs: Sequence[lp.LpConstraint]) -> lp.LpProblem:
   return reduce(iadd, constrs, lp_prob)


def head(x: Sequence) -> Any:
    return x[0]

def to_str(indnum, activity) -> str:
    return f'{indnum} - {activity}'

In [4]:
df = pd.read_csv('data/dataset.csv')

df.drop('Unnamed: 0', axis=1, inplace=True) # Drop Unnamed column

In [5]:
df.head()

,Indnum,Group,Activity,Units,Consumption,Quality_of_Life_Importance__1_10,solar_powered_water_heater,gas_water_heater,electric_water_heater_peak_hour,electric_water_heater_off_peak,gas,natural_gas,hybrid,electric_peak_hours,electric_off_peak_hours,jetfuel,TCF,TrQL
0,1,1,Household heating => 70F,hours,2,88,0.0,0.0,0.0,0.0,0.0,0.000436,0,0.0,0.0,0.0,0.000436,176
1,1,1,Household heating < 70F,hours,10,85,0.0,0.0,0.0,0.0,0.0,0.000872,0,0.0,0.0,0.0,0.000872,850
2,1,1,Use of heat pump,hours,0,50,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.0,0.0,0.0,0.000000,0
3,1,1,Use of air conditioner,hours,20,45,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.0,0.0,0.0,0.000000,900
4,1,2,shower - short,count,5,98,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.0,0.0,0.0,0.000000,490


In [6]:
indv1_df = df.drop('Group', axis=1).loc[df['Indnum'] == 2]

In [7]:
#indv1_df.head()

In [8]:
gp_model = create_prob('Wells Fargo Challenge', lp.LpMinimize)

## Decision Variables

In [9]:
indv_num = indv1_df.Indnum.unique()
actv_names = indv1_df.Activity.unique()

sources = np.array([
  "solar_powered_water_heater",
  "gas_water_heater",
  "electric_water_heater_peak_hour",
  "electric_water_heater_off_peak",
  "gas",
  "natural_gas",
  "hybrid",
  "electric_peak_hours",
  "electric_off_peak_hours",
  "jetfuel"
])

In [10]:
consumption_indexes = [
    (indv, activity)
    for indv in indv_num
    for activity in actv_names
]

C_ij = lp.LpVariable.dicts('C_ij', consumption_indexes, lowBound=0)

## Objective Function

### $Z_{min} = C_{ij} * \sum CF_{ijk}$

where 
    * i = individual
    * j = activity
    * k = source
    * _C_ is the consumption per unit of an activity

In [12]:
d_vars = []

for indv in indv_num:
    for activity in actv_names:
        total_actv_cf: np.ndarray = indv1_df.loc[indv1_df['Activity'] == activity, 'TCF'].values
        d_vars.append(C_ij[(indv, activity)] * head(total_actv_cf))


obj_fn = lp.lpSum(d_vars)

obj_fn

0.000872*C_ij_(2,_'Household_heating_<_70F') + 0.000436*C_ij_(2,_'Household_heating_=__70F') + 0.00017900000000000001*C_ij_(2,_'air_travel___large_plane') + 0.000354*C_ij_(2,_'car_trips___2__people_with_multiple_end_points') + 0.000554*C_ij_(2,_'car_trips___driver_and_self') + 0.000551*C_ij_(2,_'car_trips__self_only') + 0.00015*C_ij_(2,_'use_of__oven') + 0.000122*C_ij_(2,_'use_of_clothes_dryer') + 0.000135*C_ij_(2,_'use_of_cooking_range') + 4e-06*C_ij_(2,_'wash_up') + 0.0

## Constraints

### $C_{ij} * QL_{ij} >= TrQL_{ij}$

where 
    * i = individual
    * j = activity
    * _QL_ is the quality of life for an activity (constant)
    * _TrQL_ is the true quality of life for an activity it is computed as $C_{ij} * QL_{ij}$
     

In [13]:
tr_ql_conds = []

for indv in indv_num:
    for activity in actv_names:
        ql: np.ndarray = indv1_df.loc[indv1_df['Activity'] == activity, 'Quality_of_Life_Importance__1_10'].values
        tr_ql: np.ndarray = indv1_df.loc[indv1_df['Activity'] == activity, 'TrQL'].values
        condition = C_ij[(indv, activity)] * head(ql) >= head(tr_ql)
            
        tr_ql_conds.append(condition)

tr_ql_conds

[77*C_ij_(2,_'Household_heating_=__70F') + -539 >= 0,
 61*C_ij_(2,_'Household_heating_<_70F') + -122 >= 0,
 55*C_ij_(2,_'Use_of_heat_pump') + 0 >= 0,
 57*C_ij_(2,_'Use_of_air_conditioner') + -855 >= 0,
 85*C_ij_(2,_'shower___short') + -595 >= 0,
 74*C_ij_(2,_'shower___long_(__3_min)') + -962 >= 0,
 15*C_ij_(2,_'bath') + 0 >= 0,
 34*C_ij_(2,_'wash_up') + -1496 >= 0,
 20*C_ij_(2,_'use_of_dishwasher') + 0 >= 0,
 35*C_ij_(2,_'use_of_clothes_washer') + -280 >= 0,
 13*C_ij_(2,_'use_of_clothes_dryer') + -52 >= 0,
 0 >= 0,
 37*C_ij_(2,_'use_of__oven') + -296 >= 0,
 16*C_ij_(2,_'use_of_self_clean_feature_of_electric_oven') + -48 >= 0,
 0 >= 0,
 26*C_ij_(2,_'TV_computer_use') + -1638 >= 0,
 47*C_ij_(2,_'air_travel___large_plane') + -118628 >= 0,
 0 >= 0,
 43*C_ij_(2,_'car_trips__self_only') + -11739 >= 0,
 30*C_ij_(2,_'car_trips___driver_and_self') + -570 >= 0,
 0 >= 0,
 43*C_ij_(2,_'trips_using_public_ground_transportation') + -516 >= 0,
 49*C_ij_(2,_'bags_of_garbage_disposed') + -833 >= 0,
 0 

In [14]:
gp_model = add_obj_fn(gp_model, obj_fn)

gp_model = add_constraint(gp_model, tr_ql_conds)

In [17]:
gp_model.solve()

1

In [18]:
lp.LpStatus[gp_model.status]

'Optimal'

In [19]:
lp.value(gp_model.objective)

0.619405

In [20]:
for indv in indv_num:
    for activity in actv_names:
            ql: np.ndarray = indv1_df.loc[indv1_df['Activity'] == activity, 'Quality_of_Life_Importance__1_10'].values
            print(activity, '| Consumption: ', lp.value(C_ij[(indv, activity)]), '| Quality of Life', head(ql))

Household heating => 70F | Consumption:  7.0 | Quality of Life 77
Household heating < 70F | Consumption:  2.0 | Quality of Life 61
Use of heat pump | Consumption:  0.0 | Quality of Life 55
Use of air conditioner | Consumption:  15.0 | Quality of Life 57
shower - short | Consumption:  7.0 | Quality of Life 85
shower - long (> 3 min) | Consumption:  13.0 | Quality of Life 74
bath | Consumption:  0.0 | Quality of Life 15
wash-up | Consumption:  44.0 | Quality of Life 34
use of dishwasher | Consumption:  0.0 | Quality of Life 20
use of clothes washer | Consumption:  8.0 | Quality of Life 35
use of clothes dryer | Consumption:  4.0 | Quality of Life 13
use of cooking range | Consumption:  0.0 | Quality of Life 0
use of  oven | Consumption:  8.0 | Quality of Life 37
use of self-clean feature of electric oven | Consumption:  3.0 | Quality of Life 16
Small kitchen appliance in the home | Consumption:  None | Quality of Life 0
TV/computer use | Consumption:  63.0 | Quality of Life 26
air travel